In [1]:
import time
import random
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

# Kullanıcı ajanı ve headless mod
options = Options()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36")
# options.add_argument("--headless=new")  # İstersen headless mod aktif edebilirsin

# WebDriver başlatıcı fonksiyon
def start_driver():
    service = Service(ChromeDriverManager().install())
    return webdriver.Chrome(service=service, options=options)

# Text temizleme fonksiyonları
def clean_room_text(text):
    return text.replace('\n', '').replace(' ', '').strip() if text else 'N/A'

def clean_age_text(text):
    return text.split('\n')[0].strip() if text else 'N/A'

# Başlangıç ve bitiş sayfası
start_page = 1001
end_page = 1100

# Başlangıçta driver aç
driver = start_driver()
data = []

for page in range(start_page, end_page + 1):
    try:
        url = f'https://www.hepsiemlak.com/istanbul-satilik/daire?p31=1200000&p32=60000000&p33=1&p25=50&p26=350&page={page}'
        driver.get(url)

        # Rastgele bekleme süresi (5–12 saniye)
        time.sleep(random.randint(5, 12))

        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        listings = soup.find_all('div', class_='list-view-content')
        print(f"{page}. sayfada bulunan ilan sayısı: {len(listings)}")

        for listing in listings:
            price_tag = listing.find('span', class_='list-view-price')
            title_tag = listing.find('h3')
            room_tag = listing.find('span', class_='houseRoomCount')
            size_tag = listing.find('span', class_='squareMeter')
            age_tag = listing.find('span', class_='buildingAge')
            floor_tag = listing.find('span', class_='floortype')
            location_tag = listing.find('span', class_='list-view-location')
            status_tag = listing.find('span', class_='left')

            data.append({
                'Fiyat (TL)': price_tag.text.strip() if price_tag else 'N/A',
                'Durum': status_tag.text.strip() if status_tag else 'N/A',
                'Başlık': title_tag.text.strip() if title_tag else 'N/A',
                'Konum': location_tag.text.strip() if location_tag else 'N/A',
                'Metrekare': size_tag.text.strip() if size_tag else 'N/A',
                'Oda Sayısı': clean_room_text(room_tag.text) if room_tag else 'N/A',
                'Bina Yaşı': clean_age_text(age_tag.text) if age_tag else 'N/A',
                'Kat': floor_tag.text.strip() if floor_tag else 'N/A'
            })

        # Her 100 sayfada bir kaydet
        if page % 100 == 0 or page == end_page:
            df = pd.DataFrame(data)
            df.to_excel(f'test_{start_page}_{page}.xlsx', index=False, engine='openpyxl')
            print(f"💾 {start_page}-{page} arası veriler kaydedildi.")
            data = []

        # Her 50 sayfada bir driver'ı yeniden başlat
        if page % 50 == 0:
            driver.quit()
            driver = start_driver()

    except Exception as e:
        print(f"⚠️ {page}. sayfada hata oluştu: {e}")

# Son driver kapatma
driver.quit()


1001. sayfada bulunan ilan sayısı: 0
⚠️ 1002. sayfada hata oluştu: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=136.0.7103.114)
Stacktrace:
	GetHandleVerifier [0x00F6FC83+61635]
	GetHandleVerifier [0x00F6FCC4+61700]
	(No symbol) [0x00D905D3]
	(No symbol) [0x00D7FE20]
	(No symbol) [0x00D9DD1F]
	(No symbol) [0x00E03E8C]
	(No symbol) [0x00E1DF19]
	(No symbol) [0x00DFD096]
	(No symbol) [0x00DCC840]
	(No symbol) [0x00DCD6A4]
	GetHandleVerifier [0x011F45A3+2701795]
	GetHandleVerifier [0x011EFD26+2683238]
	GetHandleVerifier [0x0120AA6E+2793134]
	GetHandleVerifier [0x00F86945+155013]
	GetHandleVerifier [0x00F8D02D+181357]
	GetHandleVerifier [0x00F774D8+92440]
	GetHandleVerifier [0x00F77680+92864]
	GetHandleVerifier [0x00F62070+5296]
	BaseThreadInitThunk [0x7618FCC9+25]
	RtlGetAppContainerNamedObjectPath [0x774482AE+286]
	RtlGetAppContainerNamedObjectPath [0x7744827E+238]

⚠️ 1003. sayf